In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/aivle/공모전_기상청/dataset/'

Mounted at /content/drive


In [ ]:
df = pd.read_csv(path + 'train_heat.csv')
df

,Unnamed: 0,train_heat.tm,train_heat.branch_id,train_heat.ta,train_heat.wd,train_heat.ws,train_heat.rn_day,train_heat.rn_hr1,train_heat.hm,train_heat.si,train_heat.ta_chi,train_heat.heat_demand
0,1,2021010101,A,-10.1,78.3,0.5,0.0,0.0,68.2,-99.0,-8.2,281
1,2,2021010102,A,-10.2,71.9,0.6,0.0,0.0,69.9,-99.0,-8.6,262
2,3,2021010103,A,-10.0,360.0,0.0,0.0,0.0,69.2,-99.0,-8.8,266
3,4,2021010104,A,-9.3,155.9,0.5,0.0,0.0,65.0,-99.0,-8.9,285
4,5,2021010105,A,-9.0,74.3,1.9,0.0,0.0,63.5,-99.0,-9.2,283
...,...,...,...,...,...,...,...,...,...,...,...,...
499296,499297,2023123119,S,3.2,233.5,0.4,2.5,0.0,91.5,-99.0,2.8,34
499297,499298,2023123120,S,2.9,227.4,0.1,2.5,0.0,92.1,-99.0,2.7,35
499298,499299,2023123121,S,2.1,360.0,0.0,2.5,0.0,93.3,-99.0,1.4,35
499299,499300,2023123122,S,2.2,30.0,1.4,2.5,0.0,95.5,-99.0,1.3,40


In [ ]:
# ▒ 1. 데이터 불러오기 및 컬럼 정리 ▒
# df = pd.read_csv('dataset/train_heat.csv')  # CSV 파일 로드
df.drop(columns='Unnamed: 0', inplace=True)  # 저장 과정에서 자동 생성된 불필요한 인덱스 컬럼 제거
df.columns = [col.replace('train_heat.','') for col in df.columns]  # 컬럼명에서 접두어 제거 → 추후 코드 간결화 목적
print(df.columns)
df.info()  # 결측치, 데이터 타입 등 구조 확인

Index(['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si',
       'ta_chi', 'heat_demand'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499301 entries, 0 to 499300
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   tm           499301 non-null  int64  
 1   branch_id    499301 non-null  object 
 2   ta           499301 non-null  float64
 3   wd           499301 non-null  float64
 4   ws           499301 non-null  float64
 5   rn_day       499301 non-null  float64
 6   rn_hr1       499301 non-null  float64
 7   hm           499301 non-null  float64
 8   si           499301 non-null  float64
 9   ta_chi       499301 non-null  float64
 10  heat_demand  499301 non-null  int64  
dtypes: float64(8), int64(2), object(1)
memory usage: 41.9+ MB


In [ ]:
# ▒ 2. 시간 정보 처리 및 정렬 ▒
df['tm'] = pd.to_datetime(df['tm'], format='%Y%m%d%H')  # 문자열 시간 컬럼을 datetime 타입으로 변환
df = df.sort_values(['branch_id', 'tm'])  # 지점별로 시간 순 정렬 → 시간 기반 보간 및 시계열 분석에 필수

In [ ]:
# ▒ 3. 이상값 처리 (-99.0을 NaN으로 변경) 및 결측 비율 확인 ▒
df_nan = df.replace(-99.0, np.nan)  # 기상 데이터에서 -99.0은 측정 실패를 의미하므로 NaN으로 대체
df_nan.isnull().sum()  # 결측값 개수 확인
nan_ratio = df_nan.isnull().mean()  # 전체 대비 결측 비율 계산
nan_ratio.sort_values(ascending=False)


,0
si,0.466496
hm,0.079545
rn_hr1,0.038362
ws,0.037683
wd,0.037683
rn_day,0.037304
ta,0.026030
heat_demand,0.000046
ta_chi,0.000040
branch_id,0.000000


In [ ]:
# ▒ 4. 분석 대상 컬럼 선택 ▒
df_select = df_nan.copy()
df_select.drop(columns='si', inplace=True)  # 결측치 비율이 높은 'si' 컬럼은 모델에 불필요하다고 판단
df_select.dropna(subset=['heat_demand'], inplace=True)  # 타겟값 없는 행 제거 (학습 불가)

In [ ]:
# ▒ 5. 시간 기반 보간 준비 ▒
df_select = df_select.set_index('tm')  # 시간 기반 보간을 위해 인덱스를 시간으로 설정

In [ ]:
df_select.columns

In [ ]:
# ▒ 6. 기상 변수(time_interp_cols)에 대해 지점별 시간 보간 ▒
time_interp_cols = ['ta', 'ta_chi', 'ws', 'hm', 'rn_day']  # 온도, 풍속, 습도, 강수량 등 시간에 따라 연속적 변하는 변수
df_select[time_interp_cols] = df_select.groupby('branch_id')[time_interp_cols].transform(
    lambda group: group.interpolate(method='time')  # 시간 순서를 기준으로 선형 보간
)
df_select.isnull().sum()

In [ ]:
# ▒ 7. 바람 방향(wd)은 각도 데이터이므로 선형 보간 어려움 → sin, cos으로 변환하여 보간 ▒
wd_rad = np.deg2rad(df_select['wd'])  # 각도(0~360°)를 라디안으로 변환
df_select['wd_sin'] = np.sin(wd_rad)
df_select['wd_cos'] = np.cos(wd_rad)

# 지점별로 보간 → 극좌표 성분이므로 time 보간이 안정적
df_select['wd_sin'] = df_select.groupby('branch_id')['wd_sin'].transform(lambda g: g.interpolate(method='time'))
df_select['wd_cos'] = df_select.groupby('branch_id')['wd_cos'].transform(lambda g: g.interpolate(method='time'))

df_select.drop(columns=['wd'], inplace=True)  # 원래 wd 컬럼 제거

In [ ]:
# ▒ 8. 보간 결과 복사 후 결측 재확인 ▒
df_interp = df_select.copy()
df_interp.isnull().sum()

In [ ]:
# ▒ 9. rn_hr1 결측은 비가 오지 않은 것으로 간주하고 0으로 대체 ▒
df_interp['rn_hr1'] = df_interp['rn_hr1'].fillna(0)  # 1시간 강수량 측정 실패는 '강수 없음'으로 처리
df_interp.isnull().sum()

In [ ]:
# ▒ 10. 강수 이벤트 생성 변수 ▒
# rn_day_diff: 누적 강수량의 변화량 (강수 발생 감지용)
df_interp['rn_day_diff'] = df_interp.groupby('branch_id')['rn_day'].diff().fillna(0)

# rain_flag: 강수 이벤트 여부 (rn_hr1 > 0 또는 rn_day_diff > 0인 경우 1, 아니면 0)
df_interp['rain_flag'] = ((df_interp['rn_hr1'] > 0) | (df_interp['rn_day_diff'] > 0)).astype(int)

In [ ]:
# ▒ 11. 강수 여부에 따른 습도 통계 계산 ▒
# - 비 오는 날 평균 습도, 비 안 오는 날 중앙값 습도 → 이후 결측값 대체에 활용
hm_rainy = df_interp[df_interp['rain_flag'] == 1].groupby('branch_id')['hm'].mean().rename('hm_rainy_mean')
hm_dry = df_interp[df_interp['rain_flag'] == 0].groupby('branch_id')['hm'].median().rename('hm_dry_median')
hm_stats = pd.concat([hm_rainy, hm_dry], axis=1).reset_index()

In [ ]:
# ▒ 12. 지점별 습도 분포 시각화 (강수 여부별) ▒
hm_melted = hm_stats.melt(id_vars='branch_id',
                          value_vars=['hm_rainy_mean', 'hm_dry_median'],
                          var_name='Rain_Status',
                          value_name='Humidity')
hm_melted['Rain_Status'] = hm_melted['Rain_Status'].map({
    'hm_rainy_mean': 'Rainy',
    'hm_dry_median': 'Dry'
})

plt.figure(figsize=(14, 6))
sns.barplot(data=hm_melted, x='branch_id', y='Humidity', hue='Rain_Status', palette='Set2')
plt.title('Branch-wise Humidity: Rainy vs Dry Conditions')
plt.xlabel('Branch ID')
plt.ylabel('Relative Humidity (%)')
plt.ylim(0, 105)
plt.legend(title='Condition')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# ▒ 13. 결측 습도(hm) 보완 함수 정의 ▒
# rain_flag 기준으로 지점별 평균/중앙값으로 채움
rainy_dict = dict(zip(hm_stats['branch_id'], hm_stats['hm_rainy_mean']))
dry_dict = dict(zip(hm_stats['branch_id'], hm_stats['hm_dry_median']))

def fill_hm_branchwise(row):
    if pd.notnull(row['hm']):
        return row['hm']
    bid = row['branch_id']
    if row['rain_flag'] == 1:
        return rainy_dict.get(bid, 90.0)  # fallback: 90%
    else:
        return dry_dict.get(bid, 60.0)  # fallback: 60%


In [ ]:
# ▒ 14. 결측값 채우기 및 최종 확인 ▒
df_interp['hm'] = df_interp.apply(fill_hm_branchwise, axis=1)
display(df_interp)
print(df_interp.isnull().sum())

In [ ]:
# ▒ 15. 최종 습도 분포 시각화 (지점별 + rain_flag 기준) ▒
plt.figure(figsize=(14, 6))
sns.boxplot(data=df_interp, x='branch_id', y='hm', hue='rain_flag', palette='Set2')
plt.title('Relative Humidity by Rain Event and Branch')
plt.xlabel('Branch ID')
plt.ylabel('Relative Humidity (%)')
plt.ylim(0, 110)
plt.legend(title='Rain Occurred (1=Yes)')
plt.tight_layout()
plt.show()


In [ ]:
# ▒ 인덱스 정리 및 날짜 기반 파생 변수 생성 ▒

# 모델링 이전 단계에서 인덱스가 시간(tm)인 상태였다면 다시 일반 컬럼으로 되돌림
if df_interp.index.name == 'tm':
    df_interp.reset_index(inplace=True)

# 시간 관련 파생 변수 생성: 수요 패턴이 시간대/요일/월별로 다를 수 있기 때문
df_interp['hour'] = df_interp['tm'].dt.hour       # 하루 중 시간대
df_interp['month'] = df_interp['tm'].dt.month     # 월 (계절성 반영)
df_interp['weekday'] = df_interp['tm'].dt.weekday # 요일 (주말/평일 패턴 반영)
df_interp['weekend'] = df_interp['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [ ]:
df_interp['hour_sin'] = np.sin(2 * np.pi * df_interp['hour'] / 24)
df_interp['hour_cos'] = np.cos(2 * np.pi * df_interp['hour'] / 24)

In [ ]:
# ▒ 수치형 변수의 왜도 분석 ▒

# 수치형 변수만 선택 (로그 변환 및 정규성 확인용)
numeric_cols = df_interp.select_dtypes(include=['float64', 'int64', 'int32']).columns.tolist()

# 각 컬럼별 왜도 계산 (정규분포에서 벗어난 정도 측정)
from scipy.stats import skew
skew_results = {
    col: skew(df_interp[col].dropna()) for col in numeric_cols
}
skew_df = pd.DataFrame.from_dict(skew_results, orient='index', columns=['skewness'])
skew_df = skew_df.sort_values(by='skewness', ascending=False)


In [ ]:
# ▒ 왜도 시각화: 왜도 절댓값 > 1인 경우 로그 변환 후보로 간주 ▒

# 막대그래프용 정리
skew_df_sorted = skew_df.copy().reset_index()
skew_df_sorted.columns = ['Variable', 'Skewness']
skew_df_sorted['Highlight'] = skew_df_sorted['Skewness'].abs() > 1

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(data=skew_df_sorted, x='Skewness', y='Variable', hue='Highlight', dodge=False, palette={True: 'tomato', False: 'skyblue'})
plt.axvline(x=1, color='gray', linestyle='--')
plt.axvline(x=-1, color='gray', linestyle='--')
plt.title('Skewness of Numeric Variables (Log Transform Threshold = ±1)')
plt.xlabel('Skewness')
plt.ylabel('Variable')
plt.legend(title='|Skew| > 1')
plt.tight_layout()
plt.show()


In [ ]:
# ▒ 왜도 높은 변수의 분포 히스토그램 ▒

import math
n_cols = 3
n_rows = math.ceil(len(numeric_cols) / n_cols)

plt.figure(figsize=(n_cols * 5, n_rows * 3))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(n_rows, n_cols, i)
    sns.histplot(df_interp[col].dropna(), bins=50, kde=True)
    skew_val = skew_df.loc[col, 'skewness']
    plt.title(f'{col}\nSkew: {skew_val:.2f}')
    plt.xlabel(col)
    plt.ylabel('Freq')
    plt.tight_layout()
plt.suptitle('Distributions Sorted by Skewness (High → Low)', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
# ▒ 로그 변환 대상 변수 선정 ▒

# 로그 변환 기준: 왜도 > 1 이면서 음수 값이 없는 경우
log_transform_cols = [
    col for col in skew_df.index
    if skew_df.loc[col, 'skewness'] > 1 and df_interp[col].min() >= 0 and df_interp[col].nunique() > 2
]
print("로그 변환 대상:", log_transform_cols)

# 로그 변환 수행: log1p 사용 (log(x+1), 0 처리 가능)
for col in log_transform_cols:
    df_interp[f'{col}_log'] = np.log1p(df_interp[col])
    print(f"Log-transformed: {col} → {col}_log")


In [ ]:
# ▒ 로그 변환 전후 분포 비교 ▒

n_cols = 2
n_rows = len(log_transform_cols)
plt.figure(figsize=(n_cols * 6, n_rows * 3))

for i, col in enumerate(log_transform_cols):
    # 원본
    plt.subplot(n_rows, n_cols, 2*i + 1)
    sns.histplot(df_interp[col].dropna(), bins=50, kde=True)
    plt.title(f'Original: {col}\nSkew: {skew_df.loc[col, "skewness"]:.2f}')

    # 로그 변환
    plt.subplot(n_rows, n_cols, 2*i + 2)
    sns.histplot(df_interp[f'{col}_log'].dropna(), bins=50, kde=True, color='orange')
    new_skew = skew(df_interp[f'{col}_log'].dropna())
    plt.title(f'Log1p: {col}_log\nSkew: {new_skew:.2f}')

plt.suptitle('Before vs After Log Transform', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
df_interp.drop(columns=log_transform_cols, inplace=True)

In [ ]:
df_interp.columns

In [ ]:
# ▒ 시계열 기반 파생 변수 생성 ▒

df_ml = df_interp.copy()
target = 'heat_demand_log'  # 로그 변환된 타겟 변수 기준

# (1) Lag Features: 과거 시점의 수요 반영
for lag in [1, 2, 3, 6, 12, 24]:
    df_ml[f'lag{lag}'] = df_ml.groupby('branch_id')[target].transform(lambda x: x.shift(lag))

# (2) Moving Average: 이전 수요의 평균으로 패턴 반영
for window in [3, 6, 12, 24]:
    df_ml[f'ma{window}'] = df_ml.groupby('branch_id')[target].transform(
        lambda x: x.shift(1).rolling(window=window).mean()
    )

# (3) Difference Feature: 수요 변화량 반영 (변동성 고려)
df_ml['diff1'] = df_ml[target] - df_ml['lag1']
df_ml['diff3'] = df_ml[target] - df_ml['lag3']


In [ ]:
# ▒ 시계열 파생 변수 목록 및 결측 확인 ▒

# (1) diff 제거한 시계열 파생 변수 목록 생성
time_features = [col for col in df_ml.columns if ('lag' in col or 'ma' in col)]

# 지점별 결측치 개수 확인 (초기 구간은 shift/rolling 때문에 NaN 발생)
nan_counts = df_ml.groupby('branch_id')[time_features].apply(lambda x: x.isnull().sum())
pd.set_option('display.max_rows', 100)
display(nan_counts)


In [ ]:
df_ml['hour_sin']

In [ ]:
df_ml.columns

In [ ]:
selected_features = [
    'ta', 'ta_chi',
    'hm',
    'rn_day_log',
    'rn_hr1_log',
    'wd_sin', 'wd_cos',
    'ws_log',
    'lag1',
    'lag2',
    'lag3',
    'lag6',
    'lag12',
    'lag24',
    'ma3',
    'ma6',
    'ma12',
    'ma24',
    # 'diff1',
    # 'diff3',
    'hour',
    'hour_sin', 'hour_cos',
    'weekend'
]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import QuantileTransformer, StandardScaler

## B,C,D,G,H 지사에 관해 Clipping + 시즌 별

In [ ]:
df_ml['season_split'] = df_ml['month'].apply(lambda m: 0 if m in [1,2,3,12] else 1)

In [ ]:
tune_branch = ['B', 'C', 'D', 'G', 'H']
clip_dict = {}

df_ml['heat_demand'] = np.expm1(df_ml['heat_demand_log'])

for branch in tune_branch:
    for season in [0, 1]:
        subset = df_ml[(df_ml['branch_id'] == branch) & (df_ml['season_split'] == season)]
        if len(subset) >= 500:
            clip_dict[(branch, season)] = subset['heat_demand'].quantile(0.995)

# 모델 학습
branch_season_models = {}
branch_models = {}
season_metrics = []

for branch in df_ml['branch_id'].unique():
    if branch in tune_branch:
        for season in [0, 1]:
            df_sub = df_ml[(df_ml['branch_id'] == branch) & (df_ml['season_split'] == season)].dropna()
            if len(df_sub) < 500:
                continue

            X = df_sub[selected_features]
            y_real = df_sub['heat_demand']
            clip_threshold = clip_dict.get((branch, season), y_real.max())
            y_clipped_log = np.log1p(np.clip(y_real, 0, clip_threshold))

            model = LGBMRegressor(
                random_state=42,
                num_leaves=64,
                max_depth=7,
                learning_rate=0.05,
                n_estimators=200
            )
            model.fit(X, y_clipped_log)

            y_pred = model.predict(X)
            mae = mean_absolute_error(y_real, np.expm1(y_pred))
            mse = mean_squared_error(y_real, np.expm1(y_pred))
            rmse = np.sqrt(mse)

            branch_season_models[(branch, season)] = model
            season_metrics.append({
                'branch': branch,
                'season_split': 'Q1(1~3)' if season == 0 else 'Q2+(4~12)',
                'samples': len(df_sub),
                'MAE': mae,
                'RMSE': rmse
            })
    else:
        df_sub = df_ml[df_ml['branch_id'] == branch].dropna()
        if len(df_sub) < 500:
            continue

        X = df_sub[selected_features]
        y = df_sub['heat_demand_log']

        model = RandomForestRegressor(random_state=42)
        model.fit(X, y)

        y_pred = model.predict(X)
        y_real = np.expm1(y)
        mae = mean_absolute_error(y_real, np.expm1(y_pred))
        mse = mean_squared_error(y_real, np.expm1(y_pred))
        rmse = np.sqrt(mse)

        branch_models[branch] = model
        season_metrics.append({
            'branch': branch,
            'season_split': 'All',
            'samples': len(df_sub),
            'MAE': mae,
            'RMSE': rmse
        })

season_metrics_df2 = pd.DataFrame(season_metrics)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001952 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4043
[LightGBM] [Info] Number of data points in the train set: 8687, number of used features: 22
[LightGBM] [Info] Start training from score 6.042068
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

In [ ]:
print("지사별 MAE & RMSE:")
print(season_metrics_df2)

지사별 MAE & RMSE:
   branch season_split  samples        MAE       RMSE
0       A          All    26253   1.896163   2.892374
1       B      Q1(1~3)     8687  12.543623  18.174554
2       B    Q2+(4~12)    17568   6.504620  11.225692
3       C      Q1(1~3)     8687  11.351803  15.261627
4       C    Q2+(4~12)    17568   6.555286  10.232410
5       D      Q1(1~3)     8687   8.389992  12.082574
6       D    Q2+(4~12)    17566   4.929981   8.258227
7       E          All    26246   1.566394   2.280611
8       F          All    26254   1.528239   2.215180
9       G      Q1(1~3)     8687   8.387512  11.799379
10      G    Q2+(4~12)    17568   4.923880   7.883217
11      H      Q1(1~3)     8687   5.414299   8.526113
12      H    Q2+(4~12)    17568   3.503376   6.326311
13      I          All    26255   1.253577   1.826609
14      J          All    26253   1.406150   2.050838
15      K          All    26254   1.305966   1.893991
16      L          All    26255   0.494223   0.734600
17      M   

In [ ]:
season_metrics_df = pd.DataFrame(season_metrics)
season_metrics_df.sort_values(by=['branch', 'season_split'], inplace=True)
display(season_metrics_df)

## 예측을 위한 test data 구성

In [ ]:
test_origin = pd.read_csv(path+'/test_heat.csv')

In [ ]:
test_df = test_origin.copy()

In [ ]:
df_ml.columns

Index(['tm', 'branch_id', 'ta', 'hm', 'ta_chi', 'wd_sin', 'wd_cos',
       'rn_day_diff', 'rain_flag', 'hour', 'month', 'weekday', 'weekend',
       'hour_sin', 'hour_cos', 'rn_hr1_log', 'rn_day_log', 'heat_demand_log',
       'ws_log', 'lag1', 'lag2', 'lag3', 'lag6', 'lag12', 'lag24', 'ma3',
       'ma6', 'ma12', 'ma24', 'diff1', 'diff3', 'season_split', 'heat_demand',
       'year'],
      dtype='object')

In [ ]:
test_df.columns

Index(['TM', 'branch_ID', 'TA', 'WD', 'WS', 'RN_DAY', 'RN_HR1', 'HM', 'SI',
       'ta_chi', 'heat_demand'],
      dtype='object')

In [ ]:
test_df = test_df.rename(columns={
    'TM': 'tm',
    'branch_ID': 'branch_id',
    'TA': 'ta',
    'WD': 'wd',
    'WS': 'ws',
    'RN_DAY': 'rn_day',
    'RN_HR1': 'rn_hr1',
    'HM': 'hm',
    'SI': 'si',
    'ta_chi': 'ta_chi',
    'heat_demand': 'heat_demand'
})

In [ ]:
test_df.columns

Index(['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si',
       'ta_chi', 'heat_demand'],
      dtype='object')

In [ ]:
test_df['tm'] = pd.to_datetime(test_df['tm'], format='%Y%m%d%H')
test_df = test_df.sort_values(['branch_id', 'tm'])

In [ ]:
test_df = test_df.replace(-99.0, np.nan)  # 기상 데이터에서 -99.0은 측정 실패를 의미하므로 NaN으로 대체

In [ ]:
test_df.columns

Index(['tm', 'branch_id', 'ta', 'wd', 'ws', 'rn_day', 'rn_hr1', 'hm', 'si',
       'ta_chi', 'heat_demand'],
      dtype='object')

In [ ]:
test_df = test_df.set_index('tm')

In [ ]:
time_interp_cols = ['ta', 'ta_chi', 'ws', 'hm', 'rn_day']  # 온도, 풍속, 습도, 강수량 등 시간에 따라 연속적 변하는 변수
test_df[time_interp_cols] = test_df.groupby('branch_id')[time_interp_cols].transform(
    lambda group: group.interpolate(method='time')  # 시간 순서를 기준으로 선형 보간
)
test_df.isnull().sum()

,0
branch_id,0
ta,0
wd,4806
ws,0
rn_day,4
rn_hr1,4590
hm,0
si,75754
ta_chi,0
heat_demand,166915


In [ ]:
wd_rad = np.deg2rad(test_df['wd'])  # 각도(0~360°)를 라디안으로 변환
test_df['wd_sin'] = np.sin(wd_rad)
test_df['wd_cos'] = np.cos(wd_rad)

# 지점별로 보간 → 극좌표 성분이므로 time 보간이 안정적
test_df['wd_sin'] = test_df.groupby('branch_id')['wd_sin'].transform(lambda g: g.interpolate(method='time'))
test_df['wd_cos'] = test_df.groupby('branch_id')['wd_cos'].transform(lambda g: g.interpolate(method='time'))

test_df.drop(columns=['wd'], inplace=True)

In [ ]:
test_df['rn_hr1'] = test_df['rn_hr1'].fillna(0)

In [ ]:
test_df['rn_day_diff'] = test_df.groupby('branch_id')['rn_day'].diff().fillna(0)

# rain_flag: 강수 이벤트 여부 (rn_hr1 > 0 또는 rn_day_diff > 0인 경우 1, 아니면 0)
test_df['rain_flag'] = ((test_df['rn_hr1'] > 0) | (test_df['rn_day_diff'] > 0)).astype(int)

In [ ]:
hm_rainy = test_df[test_df['rain_flag'] == 1].groupby('branch_id')['hm'].mean().rename('hm_rainy_mean')
hm_dry = test_df[test_df['rain_flag'] == 0].groupby('branch_id')['hm'].median().rename('hm_dry_median')
hm_stats = pd.concat([hm_rainy, hm_dry], axis=1).reset_index()

In [ ]:
rainy_dict = dict(zip(hm_stats['branch_id'], hm_stats['hm_rainy_mean']))
dry_dict = dict(zip(hm_stats['branch_id'], hm_stats['hm_dry_median']))

def fill_hm_branchwise(row):
    if pd.notnull(row['hm']):
        return row['hm']
    bid = row['branch_id']
    if row['rain_flag'] == 1:
        return rainy_dict.get(bid, 90.0)  # fallback: 90%
    else:
        return dry_dict.get(bid, 60.0)  # fallback: 60%


In [ ]:
test_df['hm'] = test_df.apply(fill_hm_branchwise, axis=1)

In [ ]:
if test_df.index.name == 'tm':
    test_df.reset_index(inplace=True)

# 시간 관련 파생 변수 생성: 수요 패턴이 시간대/요일/월별로 다를 수 있기 때문
test_df['hour'] = test_df['tm'].dt.hour       # 하루 중 시간대
test_df['month'] = test_df['tm'].dt.month     # 월 (계절성 반영)
test_df['weekday'] = test_df['tm'].dt.weekday
test_df['weekend'] = test_df['weekday'].apply(lambda x: 1 if x in [5, 6] else 0)

In [ ]:
test_df

,tm,branch_id,ta,ws,rn_day,rn_hr1,hm,si,ta_chi,heat_demand,wd_sin,wd_cos,rn_day_diff,rain_flag,hour,month,weekday,weekend
0,2024-01-01 00:00:00,A,0.5,0.8,2.5,0.0,97.1,NaN,0.3,NaN,1.512608e-01,-0.988494,0.0,0,0,1,0,0
1,2024-01-01 01:00:00,A,0.4,1.0,0.0,0.0,96.8,NaN,0.1,NaN,9.979156e-01,-0.064532,-2.5,0,1,1,0,0
2,2024-01-01 02:00:00,A,-0.1,0.8,0.0,0.0,97.0,NaN,0.0,NaN,7.313537e-01,-0.681998,0.0,0,2,1,0,0
3,2024-01-01 03:00:00,A,-0.8,0.6,0.0,0.0,96.9,NaN,-0.2,NaN,-6.238796e-01,-0.781520,0.0,0,3,1,0,0
4,2024-01-01 04:00:00,A,0.1,1.5,0.0,0.0,97.0,NaN,-0.1,NaN,8.544588e-01,0.519519,0.0,0,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166910,2024-12-31 20:00:00,S,-1.1,0.0,0.0,0.0,45.8,NaN,-1.7,NaN,-2.449294e-16,1.000000,0.0,0,20,12,1,0
166911,2024-12-31 21:00:00,S,-1.3,0.0,0.0,0.0,48.3,NaN,-2.3,NaN,-2.449294e-16,1.000000,0.0,0,21,12,1,0
166912,2024-12-31 22:00:00,S,-2.4,0.0,0.0,0.0,60.0,NaN,-3.1,NaN,-2.449294e-16,1.000000,0.0,0,22,12,1,0
166913,2024-12-31 23:00:00,S,-3.6,0.0,0.0,0.0,65.7,NaN,-3.9,NaN,-2.449294e-16,1.000000,0.0,0,23,12,1,0


In [ ]:
test_df['hour_sin'] = np.sin(2 * np.pi * test_df['hour'] / 24)
test_df['hour_cos'] = np.cos(2 * np.pi * test_df['hour'] / 24)

In [ ]:
test_df['rn_hr1_log'] = np.log1p(test_df['rn_hr1'])
test_df['rn_day_log'] = np.log1p(test_df['rn_day'])
test_df['ws_log'] = np.log1p(test_df['ws'])

In [ ]:
test_df.drop(columns=log_transform_cols, inplace=True)

In [ ]:
# 🔹 예측 대상 열 생성
test_df['heat_demand_log'] = np.nan

# 🔹 예측 결과 저장 리스트
test_results = []

# 🔹 지사별 순차 예측
for branch in test_df['branch_id'].unique():
    print(f"📡 예측 중: Branch {branch}")

    train_branch = df_ml[df_ml['branch_id'] == branch].copy()
    test_branch = test_df[test_df['branch_id'] == branch].copy()
    df_concat = pd.concat([train_branch, test_branch], sort=False).sort_values('tm')
    df_concat.reset_index(drop=True, inplace=True)

    # 🔹 순차 예측
    for i in range(len(train_branch), len(df_concat)):
        # lag & ma 생성
        for lag in [1, 2, 3, 6, 12, 24]:
            df_concat.loc[i, f'lag{lag}'] = df_concat.loc[i - lag, 'heat_demand_log'] if i - lag >= 0 else np.nan
        for window in [3, 6, 12, 24]:
            if i - window >= 0:
                df_concat.loc[i, f'ma{window}'] = df_concat.loc[i - window:i - 1, 'heat_demand_log'].mean()
        df_concat.loc[i, 'diff1'] = df_concat.loc[i, 'heat_demand_log'] - df_concat.loc[i, 'lag1']
        df_concat.loc[i, 'diff3'] = df_concat.loc[i, 'heat_demand_log'] - df_concat.loc[i, 'lag3']

        # 🔹 예측
        input_row = df_concat.loc[i:i, selected_features]

        # 계절 분기 판단 (month 컬럼 필요)
        month = df_concat.loc[i, 'month']
        season_flag = 0 if month in [1, 2, 3, 12] else 1

        if (branch in tune_branch) and ((branch, season_flag) in branch_season_models):
            model = branch_season_models[(branch, season_flag)]
            clip_val = np.log1p(clip_dict.get((branch, season_flag), 9999))
            pred_log = model.predict(input_row)[0]
            pred_log = min(pred_log, clip_val)  # 클리핑 적용
        else:
            model = branch_models[branch]
            pred_log = model.predict(input_row)[0]

        df_concat.loc[i, 'heat_demand_log'] = pred_log

    # 🔹 결과 복원
    df_concat['heat_demand'] = np.expm1(df_concat['heat_demand_log'])
    df_pred = df_concat[df_concat['tm'].isin(test_branch['tm'])]
    test_results.append(df_pred[['tm', 'branch_id', 'heat_demand']])

# 🔹 제출 파일 생성
final_df = pd.concat(test_results).sort_values(['branch_id', 'tm'])
test_df = test_df.drop(columns=['heat_demand'], errors='ignore')
test_df = test_df.merge(final_df, on=['tm', 'branch_id'], how='left')
test_df.to_csv("250027.csv", index=False)
print("✅ 제출 파일 생성 완료: 250027.csv")


📡 예측 중: Branch A
📡 예측 중: Branch B
📡 예측 중: Branch C
📡 예측 중: Branch D
📡 예측 중: Branch E
📡 예측 중: Branch F
📡 예측 중: Branch G
📡 예측 중: Branch H
📡 예측 중: Branch I
📡 예측 중: Branch J
📡 예측 중: Branch K
📡 예측 중: Branch L
📡 예측 중: Branch M
📡 예측 중: Branch N
📡 예측 중: Branch O
📡 예측 중: Branch P
📡 예측 중: Branch Q
📡 예측 중: Branch R
📡 예측 중: Branch S
✅ 제출 파일 생성 완료: 250027.csv
